# Locations with Most Decline Due to Lockdown

####  Import packages and start the session

In [1]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 2
%extra_py_files s3://pedestrian-analysis-working-bucket/glue-job-scripts/util.py

import sys, io, util
from datetime import datetime, timedelta

from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
)

from pyspark.context import SparkContext
from pyspark.sql.functions import sum, col, rank, desc, lit, when
from pyspark.sql.window import Window
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Current idle_timeout is 2800 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 2
Extra py files to be included:
s3://pedestrian-analysis-working-bucket/glue-job-scripts/util.py
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::632753217422:role/pedestrians-analysis-notebook-role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: ea39848e-0a69-45c9-b6d3-16712af8714e
Job Type: glueetl
Applying the following

####  Create output glue table if it doesn't already exist
##### The results of this notebook will be loaded into this table

In [ ]:
BUCKET_NAME = 'pedestrian-analysis-working-bucket'
DATABASE_NAME = 'pedestrian_analysis_report'
OUTPUT_TABLE_NAME = 'report_location_declines_due_to_lockdown'

schema = StructType([
    StructField("location_name", StringType(), True),
    StructField("2019_count", IntegerType(), True),
    StructField("2022_count", IntegerType(), True),
    StructField("decline", IntegerType(), True),
    StructField("decline_percent", DoubleType(), True),
])

s3_path = f"s3://{BUCKET_NAME}/report/{OUTPUT_TABLE_NAME}/"
util.create_glue_catalog_table(DATABASE_NAME, OUTPUT_TABLE_NAME, schema, s3_path)

####  Load sensor_counts_by_day

In [ ]:
sensor_counts_df = glueContext.create_dynamic_frame.from_catalog(
    database="pedestrian_analysis_raw",
    table_name="sensor_counts"
).toDF()

sensor_counts_df.show(10, truncate=False)

####  Load sensor_reference_data

In [ ]:
sensor_reference_df = glueContext.create_dynamic_frame.from_catalog(
    database="pedestrian_analysis_raw",
    table_name="sensor_reference_data"
).toDF()

sensor_reference_df.show(10)

#### Calculate sensor counts 2019
##### Because the cutsoff at 2022-11-01, for the purpose of this analysis we are setting the end of each year to November 11

In [ ]:
sensor_counts_2019_df = sensor_counts_df \
    .filter(col('date_time') >= '2018-11-01') \
    .filter(col('date_time') < '2019-11-01') \
    .groupBy('sensor_id') \
    .agg(sum('hourly_count').alias('count_2019'))

sensor_counts_2019_df.show(10)

#### Calculate sensor counts 2022

In [ ]:
sensor_counts_2022_df = sensor_counts_df \
    .filter(col('date_time') >= '2021-11-01') \
    .filter(col('date_time') < '2022-11-01') \
    .groupBy('sensor_id') \
    .agg(sum('hourly_count').alias('count_2022'))

sensor_counts_2022_df.show(10)

#### Calculate the decline and decline percentages for each sensor

In [ ]:
sensor_decline_df = sensor_counts_2019_df \
    .join(sensor_counts_2022_df, on='sensor_id', how='inner') \
    .withColumn('decline', (col('count_2019') - col('count_2022'))) \
    .withColumn('decline_percent', ((col('count_2019') - col('count_2022')) / col('count_2019')) * 100)

sensor_decline_df.show(10)

In [ ]:
#### Join reference and select relevant columns

In [ ]:
sensor_decline_df = sensor_decline_df.join(
    sensor_reference_df,
    col("sensor_id") == col("location_id"),
    "left"
)

sensor_decline_df = sensor_decline_df.select(
    col('sensor_id'),
    col('sensor_description').alias('location_name'),
    col('count_2019').cast('int'),
    col('count_2022').cast('int'),
    col('decline'),
    col('decline_percent')
).orderBy(
    desc('decline_percent')
)

sensor_decline_df.show(100, truncate=False)

In [ ]:
util.upload_to_s3(glueContext, sensor_decline_df, s3_path)